# Integrated gradients for transformers models

In this example, we apply the integrated gradients method to two different sentiment analysis models. The first one is a pretrained sentiment analysis model from the  [transformers](https://github.com/huggingface/transformers) library. The second model is a combination of a pretrained BERT model and a simple feed forward network. The feed forward network is trained on the IMDB dataset using the BERT output embeddings as features. 

In text classification models, integrated gradients define an attribution value for each word in the input sentence. The attributions are calculated considering the integral of the model  gradients with respect to the word embedding layer along a straight path from a baseline instance $x^\prime$ to the input instance $x.$ A description of the method can be found [here](https://docs.seldon.io/projects/alibi/en/latest/methods/IntegratedGradients.html). Integrated gradients was originally proposed in Sundararajan et al., ["Axiomatic Attribution for Deep Networks"](https://arxiv.org/abs/1703.01365)

In [ ]:
import re
import os
import tensorflow as tf
import numpy as np
import torch
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
from transformers import BertTokenizerFast, TFBertModel, BertConfig
from alibi.explainers import IntegratedGradients
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam

Here we define some functions needed to process the data. For consistency with other [text examples](https://github.com/SeldonIO/alibi/blob/master/examples/integrated_gradients_imdb.ipynb) in alibi, we will use the IMDB dataset provided by keras. Since the dataset consists of reviews that are already tokenized, we need to decode each sentence and re-convert them into tokens using the BERT tokenizer.

In [ ]:
def decode_sentence(x, reverse_index):
    """Decodes the tokenized sentences from keras IMDB dataset into plain text.
    """
    # the `-3` offset is due to the special tokens used by keras
    # see https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset
    return " ".join([reverse_index.get(i - 3, 'UNK') for i in x])

def preprocess_reviews(reviews):
    """Preprocess the text.
    """
    REPLACE_NO_SPACE = re.compile("[.;:,!\'?\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

def process_sentences(sentence, 
                      tokenizer, 
                      max_len):
    """Tokenize the text sentences.
    """
    z = tokenizer(sentence, 
                  add_special_tokens = False, 
                  padding = 'max_length', 
                  max_length = max_len, 
                  truncation = True,
                  return_token_type_ids=True, 
                  return_attention_mask = True,  
                  return_tensors = 'np')
    return z

## Sentiment analysis on IMDB with fine-tuned model head.

We consider a text classifier fine-tuned on the IMDB dataset. We train a feed forward network which uses the concatenated output embeddings of a pretrained BERT model as input features. The BERT model and the trained ffn are combined to obtain an end-to-end text classifier.

It must be noted that training an end-to-end text classifier (i. e. combining the BERT model and the feed forward network before training) instead of training the feed forward network separately is likely to lead to better model performance. However, the latter approach is considerably faster and lighter. We use this approach here since performance optimization is beyond the scope of this notebook and the purpose of this example is to illustrate the integrated gradients method applied to a custom classifier.

In [ ]:
def get_embeddings(X_train, model, batch_size=50):

    args = X_train['input_ids']
    kwargs = {k:v for k, v in  X_train.items() if k != 'input_ids'}
    
    dataset = tf.data.Dataset.from_tensor_slices((args, kwargs)).batch(batch_size)
    dataset = dataset.as_numpy_iterator()
    
    embbedings = []
    for X_batch in dataset:
        args_b, kwargs_b = X_batch
        batch_embeddings = model(args_b, **kwargs_b)
        embbedings.append(batch_embeddings.last_hidden_state.numpy())

    return np.concatenate(embbedings, axis=0)

### Load and process data

Loading the IMDB dataset. 

In [ ]:
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
test_labels = y_test.copy()
train_labels = y_train.copy()
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

index = imdb.get_word_index()
reverse_index = {value: key for (key, value) in index.items()} 

### Extract embeddings for training

In order to speed up the training, the BERT embeddings are pre-extracted and used as features by the feed forward network.

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained("bert-base-uncased")
modelBert = TFBertModel.from_pretrained("bert-base-uncased", config=config)

modelBert.trainable=False

Decoding each sentence in the keras IMDB tokenized dataset to obtain the corresponding plain text.

In [ ]:
X_train, X_test = [], []
for i in range(len(x_train)):
    tr_sentence = decode_sentence(x_train[i], reverse_index)
    X_train.append(tr_sentence)
    te_sentence = decode_sentence(x_test[i], reverse_index)
    X_test.append(te_sentence)

Re-tokenizing the plain text using the BERT tokenizer.

In [ ]:
X_train = preprocess_reviews(X_train)
X_train = process_sentences(X_train, tokenizer, max_len)
X_test = preprocess_reviews(X_test)
X_test = process_sentences(X_test, tokenizer, max_len)

Extracting the BERT embeddings.

In [ ]:
train_embbedings = get_embeddings(X_train, 
                                  modelBert, 
                                  batch_size=100)
test_embbedings = get_embeddings(X_test,
                                 modelBert, 
                                 batch_size=100)

### Train model

Here we train the model head using the BERT output embeddings as features. The output embeddings are tensors of dimension 100 X 768, where each 768-dimensional vector represents a word in a sentence of 100 words. The embedding vectors are concatenated along the first dimension in order to represents a full review. The model head consists of one dense layer 128 hidden units followed by a 2 units layer with softmax activation. 

In [ ]:
dropout = 0.1
hidden_dims = 128

In [ ]:
class ModelOut(tf.keras.Model):

    def __init__(self, 
                 dropout=0.2, 
                 hidden_dims=128):
        super().__init__()
        
        self.dropout = dropout
        self.hidden_dims = hidden_dims
        
        self.flat = tf.keras.layers.Flatten()
        self.dense_1 =  tf.keras.layers.Dense(hidden_dims, 
                                              activation='relu')
        self.dropoutl = tf.keras.layers.Dropout(dropout)
        self.dense_2 = tf.keras.layers.Dense(2, 
                                             activation='softmax')

    def call(self, inputs):
        x = self.flat(inputs)
        x = self.dense_1(x)
        x = self.dropoutl(x)
        x = self.dense_2(x)
        return x
    
    def get_config(self):
        return {"dropout": self.dropout, 
                "hidden_dims": self.hidden_dims}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
model_out = ModelOut(dropout=dropout, hidden_dims=hidden_dims)

Training the model. If the model has been already trained, it can be loaded from the checkpoint directory setting `load_model=True`.

In [ ]:
load_model = False
batch_size = 128
epochs = 3

In [ ]:
filepath = './model_transformers/'  # change to desired save directory

model_out.compile(optimizer=Adam(1e-4), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

if not load_model:
    
    checkpoint_path = os.path.join(filepath, "training/cp-{epoch:04d}.ckpt")
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights every epoch
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, 
        verbose=1, 
        save_weights_only=True,
        save_freq='epoch')

    model_out.fit(train_embbedings, y_train, 
                  validation_data=(test_embbedings, y_test),
                  epochs=epochs, 
                  batch_size=batch_size,
                  callbacks=[cp_callback],
                  verbose=1)
else:
    epoch = 3
    load_path = os.path.join(filepath, f"training/cp-{epoch:04d}.ckpt")
    model_out.load_weights(load_path)

### Combine BERT and feed forward network

Here we combine the BERT model with the model head to obtain an end-to-end text classifier. 

In [ ]:
class TextClassifier(tf.keras.Model):

    def __init__(self, model_bert, model_out):
        super().__init__()
        self.model_bert = model_bert
        self.model_out = model_out

    def call(self, inputs, attention_mask=None):
        out = self.model_bert(inputs, attention_mask=attention_mask)
        out = self.model_out(out.last_hidden_state)
        return out
    
    def get_config(self):
        return {}

    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
text_classifier = TextClassifier(modelBert, model_out)

### Calculate integrated gradients

We pick the first 10 sentences from the test set as examples.

In [ ]:
z_test_sample = [decode_sentence(x_test[i], reverse_index) for i in range(10)]
z_test_sample = preprocess_reviews(z_test_sample)
z_test_sample = process_sentences(z_test_sample, tokenizer, max_len)

x_test_sample = z_test_sample['input_ids']
kwargs = {k:v for k,v in z_test_sample.items() if k == 'attention_mask'}

We calculate the attributions with respect to the first embedding layer of the BERT encoder.

In [ ]:
bl = text_classifier.layers[0].bert.encoder.layer[0]

In [ ]:
n_steps = 5
method = "gausslegendre"
internal_batch_size = 5
ig  = IntegratedGradients(text_classifier,
                          layer=bl,
                          n_steps=n_steps, 
                          method=method,
                          internal_batch_size=internal_batch_size)

In [ ]:
predictions = text_classifier(x_test_sample, **kwargs).numpy().argmax(axis=1)
explanation = ig.explain(x_test_sample, 
                         forward_kwargs=kwargs,
                         baselines=None, 
                         target=predictions)

In [ ]:
# Get attributions values from the explanation object
attrs = explanation.attributions[0]
print('Attributions shape:', attrs.shape)

In [ ]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

In [ ]:
i = 1
x_i = x_test_sample[i]
attrs_i = attrs[i]
pred = predictions[i]
pred_dict = {1: 'Positive review', 0: 'Negative review'}

In [ ]:
from IPython.display import HTML
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"

In [ ]:
def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    import matplotlib as mpl
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)
    
    # now compute hex values of colors
    colors = list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))
    return colors

In [ ]:
words = tokenizer.decode(x_i).split()
colors = colorize(attrs_i)

In [ ]:
print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))

In [ ]:
HTML("".join(list(map(hlstr, words, colors))))